In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#read csv file
data = pd.read_csv('data/stat_acc_V3_cleared.csv', sep=';')


### Suppression des colonnes inutiles et modificaiton format date

In [155]:
#replacer le format date 'YYYYMMDD HH:MM' en format numeric 'YYYYMMDDHHMM'
data['date'] = data['date'].str.replace(':','')
data['date'] = data['date'].str.replace('-','')
data['date'] = data['date'].str.replace(' ','')

#convertir la colonne date en numeric
data['date'] = pd.to_numeric(data['date'])

new_data = data

#supression colonne num_veh
new_data = new_data.drop('num_veh', axis=1)

#drop colonne ville
new_data = new_data.drop('ville', axis=1)

#drop colonne id_code_insee
new_data = new_data.drop('id_code_insee', axis=1)

#drop colonne Num_Acc
new_data = new_data.drop('Num_Acc', axis=1)

#drop colonne id_usa
new_data = new_data.drop('id_usa', axis=1)


### Réduction de dimension

In [156]:
# Identifier les attributs fortement corrélés

# Créer une matrice de corrélation
corr_matrix = new_data.corr().abs()

# Identifiez les paires d'attributs fortement corrélés
threshold = 0.8  # Définissez votre propre seuil de corrélation
highly_correlated_pairs = []

# Parcourir la matrice de corrélation
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) >= threshold:
            pair = (corr_matrix.columns[i], corr_matrix.columns[j])
            highly_correlated_pairs.append(pair)

#verifier les paires d'attributs fortement corrélés
# print(highly_correlated_pairs)

# Afficher les paires d'attributs fortement corrélés
for pair in highly_correlated_pairs:
    print("Attribut 1:", pair[0])
    print("Attribut 2:", pair[1])

    #suppression d'une des deux colonnes fortement corrélées (ici on supprime la colonne 'an_nais' jugée moins pertinente)
    new_data = data.drop(pair[0], axis=1)

    print()


Attribut 1: an_nais
Attribut 2: age

